In [16]:
import csv
from datetime import date, datetime
import requests
from pprint import pprint

In [17]:
def get_divisions():
    return requests.get(f"http://localhost:8080/core/v1/tournament-divisions?expand=tournament").json()

In [18]:
def reset_topics(ids):
    for id in ids:
        requests.patch(f"http://localhost:8080/core/v1/tournament-divisions/{id}", json={
            'topic': {
                'disconnect': True
            }
        })

In [19]:
divisions = get_divisions()

reset_topics(list(map(lambda d: d['id'], divisions)))

with open("topics.csv", "r") as f:
    table = csv.reader(f)
    rows = list(table)[1:]
    rows.reverse()

    resolution_2_id = {}
    updated_division_ids = []

    for row in rows:
        data = requests.post("http://localhost:8080/core/v1/topics", json={
            'resolution': row[1],
            'event': 'PublicForum',
            'nickname': "",
            'tags': {
                'connectOrCreate': list(map(lambda t: {
                    'where': {
                        'tag': t
                    },
                    'create': {
                        'tag': t
                    }
                }, row[2].split(';')))
            }
        }).json()

        resolution_2_id[row[1]] = data['id']

    for row in rows:
        (m, y) = row[0].split('/')
        topic_start = date(year=int(y) + 2000, month=int(m), day=1)

        for division in divisions:
            if division['topicId'] is not None or division['id'] in updated_division_ids: continue
            tourn_start = datetime.strptime(division['tournament']['start'], "%Y-%m-%dT%H:%M:%S.%f%z").date()

            if topic_start <= tourn_start:
                requests.patch(f"http://localhost:8080/core/v1/tournament-divisions/{division['id']}", json={
                    'topic': {
                        'connect': {
                            'id': resolution_2_id[row[1]]
                        }
                    }
                })
                updated_division_ids.append(division['id'])